# 비어플 4주차 - 모델링 전처리
#### 2022110484 장윤서

## 사용 모델: 랜덤포레스트 
랜덤포레스트 공부

### 랜덤포레스트란?


✅ 랜덤포레스트는 앙상블 학습 방법 중 하나로 여러 결정트리를 조합하여 더 강력한 분류 모델을 구축하는 방법이다.

</aside>

따라서 랜덤포레스트에 대해 알기 전에 앙상블 기법과 그 종류를 알아보도록 하자 

### 앙상블 기법

- 여러 개의 모델을 생성하고 각 모델들의 예측 결과를 결합하여 정확한 예측을 도출하는 기법
- 여러 약한 분류기를 결합하여 강한 분류기를 생성
- 의사결정 트리를 기본 알고리즘으로 사용함

### 배깅

:여러 모델을 독립적으로 학습시킨 후 그 결과를 투표 혹은 평균을 통해 종합하는 방식 

- 데이터 셋에서 부트스트랩 방식을 이용하여 여러 개의 데이터셋을 뽑아서 모델에 할당 후 각 모델 학습 시킴
- 각 결정트리가 예측값을 출력하고 그 값들을 집계하여 최종 결과값 예측
- 병렬방식
- 대표 모델: 랜덤포레스트


### 랜덤포레스트

- 랜덤 포레스트의 알고리즘 과정은 모집단으로부터 표집한 학습 데이터에서 부트스트랩 표본을 생성한다. 이런 과정을 반복하여 n개의 부트스트랩 데이터를 생성하여 의사결정나무 알고리즘을 적용해 랜덤하게 m개의 예측변수를 선택하게 된다

- 이렇게 선택한 예측변수를 이용하여 훈련목적 함수를 최대로 만드는 분할을 찾아 노드 분할 함수의 최적값을 구한다. 빅데이터에서 수천 개의 예측변수를 활용할 수 있다는 장점이 있으며 예측 중심 전략이기 때문에 중요한 변수를 찾아내는데 효율적인 분석 방법이다.

- 하지만 랜덤 포레스트 분석 결과에 대한 이론적인 설명이나 해석이 어렵다는 단점을 가지고 있다

## 모델링 전처리 

1. 범주형 자료 수치화 하기 

Seasons , Holiday, Functional Day 변수가 범주형인데 해당 변수들의 경우 명목형 변수이고 범주가 작아서 원핫 인코딩을 해도 될 거 같았다. 그러나 선형 계열의 모델이 아닌 경우 라벨 인코딩을 사용해도 된다고 하여 변수의 개수를 유지하는 라벨 인코딩을 사용 

2. 변수 스케일링 

랜덤 포레스트나 트리 기반 모델은 변수를 스케일링 할 필요가 없다고 하여서 진행하지 않기로 함 

3. 변수 선택

랜덤 포레스트의 경우 전체 feature 을 모두 사용하는데 이는 특성의 일부를 랜덤하게 선택하여 결정트리를 훈련하기 때문이다 

즉  하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 train 되어 과대적합을 줄이고 일반화 성능을 높이는 데 도움을 준다.  

따라서 랜덤포레스트의 경우 변수를 선택할 필요는 없는 모델인 거 같아서 생략 하기로 했다

4. 검증과정 

이미 데이터가 train 과 test 로 나뉘어져있는 데이터라서 test 에서 valid 데이터만 나눠서 사용하면 될 거 같다 
train 의 개수가 8040 개 , test 의 개수가 721개 이므로 

7:2:1 의 비율로 나눠서 진행한다

# 라이브러리 설치하기

In [1]:
#기본 라이브러리 설치
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format = 'retina'  #plot내 글씨를 선명하게 해주는 옵션

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler

# 데이터 불러오기

In [36]:
bike_clean=pd.read_csv("Bike_clean.csv")

In [37]:
df_clean=bike_clean.copy()

In [38]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8040 entries, 0 to 8039
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Date                         8040 non-null   object 
 1   Rented Bike Count            8040 non-null   int64  
 2   Hour                         8040 non-null   int64  
 3   Temperature(ì§¸C)            8040 non-null   float64
 4   Humidity(%)                  8040 non-null   int64  
 5   Wind speed (m/s)             8040 non-null   float64
 6   Visibility (10m)             8040 non-null   int64  
 7   Dew point temperature(ì§¸C)  8040 non-null   float64
 8   Solar Radiation (MJ/m2)      8040 non-null   float64
 9   Rainfall(mm)                 8040 non-null   float64
 10  Snowfall (cm)                8040 non-null   float64
 11  Seasons                      8040 non-null   object 
 12  Holiday                      8040 non-null   object 
 13  Functioning Day   

In [39]:
df_clean

,Date,Rented Bike Count,Hour,Temperature(ì§¸C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(ì§¸C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,month,Day_of_Week
0,2017-12-01,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday
1,2017-12-01,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday
2,2017-12-01,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday
3,2017-12-01,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday
4,2017-12-01,78,4,-6.0,36,1.6,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,2018-10-31,1507,19,8.4,53,1.9,2000,-0.6,0.0,0.0,0.0,Autumn,No Holiday,Yes,10,Wednesday
8036,2018-10-31,1176,20,7.6,59,0.7,2000,0.0,0.0,0.0,0.0,Autumn,No Holiday,Yes,10,Wednesday
8037,2018-10-31,1069,21,7.6,59,3.0,1929,0.0,0.0,0.0,0.0,Autumn,No Holiday,Yes,10,Wednesday
8038,2018-10-31,1088,22,6.8,58,2.2,1936,-0.9,0.0,0.0,0.0,Autumn,No Holiday,Yes,10,Wednesday


In [40]:
#날짜변수 변환하기 .
df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%Y-%m-%d')

# 'month' 컬럼에 대해 날짜 형식 변환
df_clean['month'] = pd.to_datetime(df_clean['month'], format='%m').dt.month


In [41]:
# 범주형 변수 변환하기 
df_clean['Seasons']=df_clean['Seasons'].astype('category')
df_clean['Holiday']=df_clean['Holiday'].astype('category')
df_clean['Functioning Day']=df_clean['Functioning Day'].astype('category')
#요일 변수 범주형 처리하기 
df_clean['Day_of_Week']=df_clean['Day_of_Week'].astype('category')

In [42]:
df_clean.head()

,Date,Rented Bike Count,Hour,Temperature(ì§¸C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(ì§¸C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,month,Day_of_Week
0,2017-12-01,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday
1,2017-12-01,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday
2,2017-12-01,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday
3,2017-12-01,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday
4,2017-12-01,78,4,-6.0,36,1.6,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes,12,Friday


# 범주형 데이터 수치화 하기 

In [43]:
# 'category' 타입의 열 선택
categorical_cols = df_clean.select_dtypes(include=['category']).columns.tolist()

# 라벨 인코더 객체 생성 및 적용
label_encoder = LabelEncoder()
for col in categorical_cols:
   
    df_clean[col] = label_encoder.fit_transform(df_clean[col])

# 결과 확인
print(df_clean[categorical_cols])


      Seasons  Holiday  Functioning Day  Day_of_Week
0           3        1                1            0
1           3        1                1            0
2           3        1                1            0
3           3        1                1            0
4           3        1                1            0
...       ...      ...              ...          ...
8035        0        1                1            6
8036        0        1                1            6
8037        0        1                1            6
8038        0        1                1            6
8039        0        1                1            6

[8040 rows x 4 columns]


In [44]:
df_clean

,Date,Rented Bike Count,Hour,Temperature(ì§¸C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(ì§¸C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,month,Day_of_Week
0,2017-12-01,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,3,1,1,12,0
1,2017-12-01,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,3,1,1,12,0
2,2017-12-01,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,3,1,1,12,0
3,2017-12-01,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,3,1,1,12,0
4,2017-12-01,78,4,-6.0,36,1.6,2000,-18.6,0.0,0.0,0.0,3,1,1,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,2018-10-31,1507,19,8.4,53,1.9,2000,-0.6,0.0,0.0,0.0,0,1,1,10,6
8036,2018-10-31,1176,20,7.6,59,0.7,2000,0.0,0.0,0.0,0.0,0,1,1,10,6
8037,2018-10-31,1069,21,7.6,59,3.0,1929,0.0,0.0,0.0,0.0,0,1,1,10,6
8038,2018-10-31,1088,22,6.8,58,2.2,1936,-0.9,0.0,0.0,0.0,0,1,1,10,6


# Train , Valid, Test 셋 분리

In [45]:
#X(독립변수), y(종속변수) 분리
X = df_clean.drop("Rented Bike Count", axis=1)
y = df_clean["Rented Bike Count"]

In [50]:
#train과 valid 셋 분리

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=100)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(6432, 15) (1608, 15) (6432,) (1608,)


# 2차 전처리 데이터 

In [51]:
df_clean.to_csv('Bike_clean2.csv', index=False)